# Import necessary libraries

In [1]:
import numpy as np
from scipy.fftpack import dct, idct
from PIL import Image
import pickle
from io import BytesIO
import matplotlib.pyplot as plt
import heapq
from collections import Counter, defaultdict
import os
import gzip

# Load the image

In [2]:
def load_image(filepath):
    image = Image.open(filepath).convert('L')
    return np.array(image)

# DCT and IDCT

In [3]:
# DCT and inverse DCT for 8x8 blocks
def compute_dct(block):
    return dct(dct(block.T, norm='ortho').T, norm='ortho')

def compute_idct(block):
    return idct(idct(block.T, norm='ortho').T, norm='ortho')

# Quantization Matrix

In [4]:
# Standard JPEG quantization matrix
quantization_matrix = np.array([
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 58, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
])

# Huffman tree

In [5]:
# Huffman Tree generation
def build_huffman_tree(data):
    frequency = Counter(data)
    heap = [[weight, [symbol, ""]] for symbol, weight in frequency.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        lo = heapq.heappop(heap)
        hi = heapq.heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])

    return {symbol: code for symbol, code in heapq.heappop(heap)[1:]}

def huffman_encode(data, huffman_table):
    return ''.join(huffman_table[val] for val in data)

def huffman_decode(encoded_data, huffman_table):
    inverse_table = {code: symbol for symbol, code in huffman_table.items()}
    decoded_data, current_code = [], ""
    for bit in encoded_data:
        current_code += bit
        if current_code in inverse_table:
            decoded_data.append(inverse_table[current_code])
            current_code = ""
    return decoded_data

# Quantization

In [6]:
# Quantize the DCT block
def quantize_block(dct_block, quant_matrix, quality_factor):
    # Ensure quality_factor is within valid bounds
    quality_factor = max(1, min(100, quality_factor))
    #print(dct_block.shape)

    if quality_factor < 50:
      scale = 5000 / quality_factor
    else:
      scale = 200 - 2 * quality_factor

    adjusted_matrix = np.floor((quantization_matrix * scale + 50) / 100)
    adjusted_matrix[adjusted_matrix == 0] = 1
    #print(adjusted_matrix.shape)
    # Quantize the DCT block
    quantized_block = np.round(dct_block / adjusted_matrix).astype(int)
    # print(adjusted_matrix)
    return quantized_block, adjusted_matrix

def dequantize_block(quantized_block, adjusted_matrix):
    return (quantized_block * adjusted_matrix).astype(float)

# ZigZag Order

In [7]:
def zigzag_order(block):
    rows, cols = block.shape
    solution = [[] for i in range(rows + cols - 1)]

    for row in range(rows):
        for col in range(cols):
            sum = row + col
            if(sum % 2 == 0):
                #add at beginning
                solution[sum].insert(0, block[row][col])
            else:
                #add at end of the list
                solution[sum].append(block[row][col])

    return np.array([item for sublist in solution for item in sublist])
def inverse_zigzag_order(zigzag_values, block_size=8):
    matrix = np.zeros((block_size, block_size), dtype=int)
    index = 0

    for i in range(block_size + block_size - 1):
        if i % 2 == 0:  # Even diagonal (up-right)
            # Moving up-right with decrementing row and incrementing column
            for j in range(min(i + 1, block_size)):
                row = i - j
                col = j
                if row >= 0 and row < block_size and col >= 0 and col < block_size:  # Check matrix boundaries
                    matrix[row, col] = zigzag_values[index]
                    index += 1
        else:  # Odd diagonal (down-left)
            # Moving down-left with incrementing row and decrementing column
            for j in range(min(i + 1, block_size)):
                col = i - j
                row = j
                if row >= 0 and row < block_size and col >= 0 and col < block_size:
                    matrix[row, col] = zigzag_values[index]
                    index += 1

    return matrix

# RLE

In [8]:
def run_length_encoding(zigzag_values,huffman_table):
    encoded_values = []
    run_length = 0

    for value in zigzag_values:
        if value == 0:
            run_length += 1
        else:
            encoded_values.append((np.uint8(run_length),huffman_table[value]))  # Store run-length, size, and value
            run_length = 0  # Reset run-length

    # Store trailing zeros count in the EOB marker:
    encoded_values.append((np.uint8(run_length),-1))  # EOB with trailing zeros count
    # encoded_values.append((0, 0, 'EOB'))
    return encoded_values

def run_length_decoding(encoded_values):
    decoded_values = []
    trailing_zeros_count = 0  # Initialize trailing zeros count

    for run_length , value in encoded_values:
        if value == -1:  # Check for EOB marker
            trailing_zeros_count = value  # Get trailing zeros count from EOB
            break  # Stop decoding at the EOB marker
        else:
            decoded_values.extend([0] * run_length)  # Add zeros for the run-length
            decoded_values.append(value)  # Append the actual value

    # Add trailing zeros (if any) after decoding:
    decoded_values.extend([0] * trailing_zeros_count)

    return decoded_values

# Padding image

In [9]:
def pad_image(image, block_size=8):
    """Pad the image to make its dimensions a multiple of block_size."""
    height, width = image.shape
    pad_height = (block_size - (height % block_size)) % block_size
    pad_width = (block_size - (width % block_size)) % block_size
    padded_image = np.pad(image, ((0, pad_height), (0, pad_width)), mode='constant')
    return padded_image, height, width

# Compress Image PCA

In [10]:
def pca_compress(image, num_components):
    # Center the image matrix
    mean_image = np.mean(image, axis=0)
    centered_image = image - mean_image

    # Compute the covariance matrix
    covariance_matrix = np.cov(centered_image, rowvar=False)

    # Eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

    # Sort eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Select top principal components
    selected_eigenvectors = eigenvectors[:, :num_components]

    # Project image onto the selected principal components
    compressed_image = np.dot(centered_image, selected_eigenvectors)

    return compressed_image, mean_image, selected_eigenvectors

# Compress image using PCA
def compress_image_pca(filepath, num_components):
    image = load_image(filepath)
    compressed_image, mean_image, selected_eigenvectors = pca_compress(image, num_components)

    # Save compressed data
    compressed_data = {
        'compressed_image': compressed_image,
        'mean_image': mean_image,
        'eigenvectors': selected_eigenvectors
    }
    compressed_stream = BytesIO()
    with gzip.GzipFile(fileobj=compressed_stream, mode='wb') as gzip_file:
        pickle.dump(compressed_data, gzip_file)

    return compressed_stream.getvalue()


# Decompress Image PCA

In [11]:
def pca_decompress(compressed_image, mean_image, selected_eigenvectors):
    # Reconstruct the image from principal components
    reconstructed_image = np.dot(compressed_image, selected_eigenvectors.T) + mean_image
    return reconstructed_image

# Decompress image using PCA
def decompress_image_pca(compressed_data):
    """Decompresses PCA-compressed image data and reconstructs the image."""
    compressed_stream = BytesIO(compressed_data)
    # Decompress the data using gzip and load it using pickle
    with gzip.GzipFile(fileobj=compressed_stream, mode='rb') as gzip_file:
        decompressed_data = pickle.load(gzip_file)
    # Extract the decompressed components
    decompressed_image = decompressed_data['compressed_image']
    mean_image = decompressed_data['mean_image']
    selected_eigenvectors = decompressed_data['eigenvectors']
    # Reconstruct the image from the PCA components
    reconstructed_image = pca_decompress(decompressed_image, mean_image, selected_eigenvectors)
    # Ensure pixel values are within valid range (0-255) and return as uint8
    return np.clip(reconstructed_image, 0, 255).astype(np.uint8)

# Compress Image JPEG

In [12]:
def compress_image(filepath, quality_factor):
    image = load_image(filepath)
    height, width = image.shape
    compressed_data = []
    dc_diff_values = []
    ac_values = []

    adjusted_matrix = None
    quantized_blocks = []
    dct_blocks = []

    #pad the image so that both row and column are nearest multiple 8
    padded_image, original_height, original_width = pad_image(image, 8)
    padded_height, padded_width = padded_image.shape
    
    # Divide into 8x8 blocks, apply DCT and quantize
    previous_dc = 0
    #store the orignal size of image as well
    for i in range(0, padded_height, 8):
        for j in range(0, padded_width, 8):
            dct_blocks.append(padded_image[i:i+8, j:j+8])
            block = padded_image[i:i+8, j:j+8]
            dct_block = compute_dct(block)
   
            quantized_block, adjusted_matrix = quantize_block(dct_block, quantization_matrix, quality_factor)
            quantized_blocks.append(quantized_block)

            # DC and AC separation
            dc_diff = quantized_block[0, 0] - previous_dc
            previous_dc = quantized_block[0, 0]
            dc_diff_values.append(dc_diff)
            ac_values.extend(zigzag_order(quantized_block)[1:])

    # Build Huffman trees
    dc_huffman_table = build_huffman_tree(dc_diff_values)
    ac_huffman_table = build_huffman_tree([val for val in ac_values if val != 0])
   
    # Compress each block
    previous_dc = 0
    for i in range(0, padded_height, 8):
        for j in range(0, padded_width, 8):
            block = padded_image[i:i+8, j:j+8]
            dct_block = compute_dct(block)
            quantized_block, adjusted_matrix = quantize_block(dct_block, quantization_matrix, quality_factor)

            # Encode DC
            dc_diff = quantized_block[0, 0] - previous_dc
            previous_dc = quantized_block[0, 0]
            dc_encoded = dc_huffman_table[dc_diff]

            # Encode AC with run-length, size, and Huffman code
            zigzag_ac = zigzag_order(quantized_block)[1:]
            ac_encoded = run_length_encoding(zigzag_ac,ac_huffman_table)
            compressed_data.append((dc_encoded, ac_encoded))

    header = {
        'file_size': (height, width),
        'dc_huffman_table': dc_huffman_table,
        'ac_huffman_table': ac_huffman_table,
        'adjusted_matrix': adjusted_matrix
    }

    compressed_stream = BytesIO()
    with gzip.GzipFile(fileobj=compressed_stream, mode='wb') as f:
        pickle.dump(header, f)
        pickle.dump(compressed_data, f)

    return compressed_stream.getvalue(), dc_huffman_table, ac_huffman_table

# Decompress Image JPEG

In [13]:
def decompress_image(compressed_data):

    compressed_stream = BytesIO(compressed_data)

    # Open the gzip file in read mode
    with gzip.GzipFile(fileobj=compressed_stream, mode='rb') as f:
        # Load the header and compressed data from the stream
        header = pickle.load(f)
        compressed_blocks = pickle.load(f)
    
    # Extract header information
    original_height, original_width = header['file_size']  # Extract original dimensions
    dc_huffman_table = header['dc_huffman_table']
    ac_huffman_table = header['ac_huffman_table']
    adjusted_matrix = header['adjusted_matrix']
    
    # Initialize decompressed image with padded size
    blocks_per_row = int(np.sqrt(len(compressed_blocks)))  
    block_size = 8
    decompressed_image = np.zeros((original_height, original_width), dtype=np.uint8)
    decompressed_image = pad_image(decompressed_image, block_size)[0]
    padded_height, padded_width = decompressed_image.shape

    # Initialize previous DC coefficient to 0
    previous_dc = 0
    idx = 0
    dequantized_blocks = []
    inverse_dct = []

    # Decompress each block
    for i in range(0, padded_height, block_size):
        for j in range(0, padded_width, block_size):
            dc_encoded, ac_encoded = compressed_blocks[idx]

            # Decode DC coefficient
            dc_diff = huffman_decode(dc_encoded, dc_huffman_table)[0]
            dc_coeff = previous_dc + dc_diff
            previous_dc = dc_coeff

            # Decode AC coefficients
            ac_decoded = []
            trailing_zeros = 0
            for run_length, value in ac_encoded:
                if value == -1:
                    trailing_zeros = run_length
                    break
                else:
                    zeros = [0] * run_length
                    decoded_value = huffman_decode(value, ac_huffman_table)[0]
                    ac_decoded.extend(zeros + [decoded_value])

            # Pad decoded AC coefficients with zeros if necessary and add trailing zeros
            ac_decoded.extend([0] * (63 - len(ac_decoded)))
            ac_decoded.extend([0] * trailing_zeros)

            # Insert DC coefficient at the beginning of ac_decoded
            ac_decoded.insert(0, dc_coeff)

            # Reconstruct the 8x8 block using inverse zigzag
            quantized_block = inverse_zigzag_order(ac_decoded, block_size=8)

            # Dequantize the block
            dct_block = dequantize_block(quantized_block, adjusted_matrix)
            dequantized_blocks.append(dct_block)

            # Apply inverse DCT
            decompressed_block = compute_idct(dct_block)
            inverse_dct.append(decompressed_block)

            # Clip pixel values to valid range and assign to decompressed image
            decompressed_image[i:i + 8, j:j + 8] = np.clip(decompressed_block, 0, 255)
            idx += 1

    # Crop the decompressed image to its original size
    cropped_image = decompressed_image[:original_height, :original_width]

    # Save intermediate results for debugging
    with open('dequantized_dct.txt', 'w') as f:
        for block in dequantized_blocks:
            np.savetxt(f, block, fmt='%d', header="Block:")
    with open('inverse_dct.txt', 'w') as f:
        for block in inverse_dct:
            np.savetxt(f, block, fmt='%d', header="Block:")

    return cropped_image

# Show Images

In [14]:
 # Display Images
def show_images(original_image, decompressed_image, decompressed_pca, save_path):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))  # Create 3 axes for 3 images
    axes[0].imshow(original_image, cmap='gray')
    axes[0].set_title("Original Image")
    axes[1].imshow(decompressed_image, cmap='gray')
    axes[1].set_title("Compressed Image (JPEG)")
    axes[2].imshow(decompressed_pca, cmap='gray')
    axes[2].set_title("Compressed Image (PCA)")

    # Remove axis ticks for better image viewing
    for ax in axes:
        ax.axis('off')

    # Save the figure to a file
    plt.savefig(save_path)
    plt.close()  # Close the figure to free memory


In [15]:
# Calculate RMSE between two images
def calculate_rmse(original_image, decompressed_image):
    """Calculates the RMSE between two images."""
    return np.sqrt(np.mean((original_image - decompressed_image) ** 2))

# Calculate BPP (Bits per Pixel)
def calculate_bpp(compressed_data, image_shape):
    """Calculates the BPP of a compressed image."""
    compressed_size_bits = len(compressed_data) * 8  # Size in bits
    num_pixels = image_shape[0] * image_shape[1]
    return compressed_size_bits / num_pixels

In [16]:
def compare_all(folder_path, quality_factor, save_base_folder):
    print(f"\nProcessing folder: {folder_path}")
    dataset_name = os.path.basename(folder_path.strip('/'))  # Extract dataset name (e.g., 'chair')
    
    # Create a folder for saving comparison images if it doesn't exist
    save_folder = os.path.join(save_base_folder, dataset_name)
    os.makedirs(save_folder, exist_ok=True)
    
    # For plotting RMSE vs BPP and Compression Rate for both PCA and JPEG
    rmse_values_jpeg = []
    bpp_values_jpeg = []
    rmse_values_pca = []
    bpp_values_pca = []
    compression_rates_jpeg = []
    compression_rates_pca = []
    image_labels = []  # To track filenames for compression rate plot

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            filepath = os.path.join(folder_path, filename)
            
            try:
                # Load original image
                original_image = load_image(filepath)
                
                # Compress and decompress image (JPEG compression)
                compressed_data_jpeg, dc_huffman_table, ac_huffman_table = compress_image(filepath, quality_factor)
                decompressed_image_jpeg = decompress_image(compressed_data_jpeg)
                
                # Compress and decompress image (PCA-based compression)
                compressed_data_pca = compress_image_pca(filepath, quality_factor)
                decompressed_image_pca = decompress_image_pca(compressed_data_pca)  # Use compressed_data_pca
                
                # Calculate RMSE and BPP for JPEG
                rmse_jpeg = calculate_rmse(original_image, decompressed_image_jpeg)
                bpp_jpeg = calculate_bpp(compressed_data_jpeg, original_image.shape)
                
                # Calculate RMSE and BPP for PCA
                rmse_pca = calculate_rmse(original_image, decompressed_image_pca)
                bpp_pca = calculate_bpp(compressed_data_pca, original_image.shape)
                
                # Calculate compression rate (original vs. compressed size)
                original_size = original_image.nbytes  # Get original file size in bytes
                
                # For JPEG
                compressed_size_jpeg = len(compressed_data_jpeg)  # Get the compressed data size for JPEG
                compression_rate_jpeg = original_size / compressed_size_jpeg
                
                # For PCA
                compressed_size_pca = len(compressed_data_pca)  # Get the compressed data size for PCA
                compression_rate_pca = original_size / compressed_size_pca

                # Store RMSE, BPP, and compression rates for both JPEG and PCA
                rmse_values_jpeg.append(rmse_jpeg)
                bpp_values_jpeg.append(bpp_jpeg)
                compression_rates_jpeg.append(compression_rate_jpeg)
                
                rmse_values_pca.append(rmse_pca)
                bpp_values_pca.append(bpp_pca)
                compression_rates_pca.append(compression_rate_pca)
                
                image_labels.append(filename)
                
                # Generate a unique output file path based on the folder and image name
                output_filename = f"{os.path.splitext(filename)[0]}_comparison+{quality_factor}.png"
                output_path = os.path.join(save_folder, output_filename)
                
                # Visualize and save the images
                show_images(original_image, decompressed_image_jpeg, decompressed_image_pca, output_path)
                
                print(f"Saved comparison for {filename} as {output_filename}")
                
            except Exception as e:
                print(f"Error processing {filename}: {e}")
    
    # After processing all images in the folder, generate RMSE vs BPP plots for both PCA and JPEG
    plt.figure(figsize=(8, 6))
    plt.scatter(bpp_values_jpeg, rmse_values_jpeg, marker='o', color='blue', label=f'JPEG (Quality Factor: {quality_factor})')
    plt.scatter(bpp_values_pca, rmse_values_pca, marker='x', color='red', label=f'PCA (Quality Factor: {quality_factor})')
    plt.xlabel("BPP (Bits per Pixel)")
    plt.ylabel("RMSE (Root Mean Squared Error)")
    plt.title(f"RMSE vs BPP (JPEG vs PCA) - Quality Factor: {quality_factor}")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(save_folder, f"rmse_vs_bpp_{quality_factor}.png"))
    plt.close()

    # Generate Compression Rate plots for both JPEG and PCA
    plt.figure(figsize=(8, 6))
    plt.bar(image_labels, compression_rates_jpeg, color='blue', label='JPEG')
    plt.bar(image_labels, compression_rates_pca, color='red', alpha=0.5, label='PCA')
    plt.xlabel("Image")
    plt.ylabel("Compression Rate")
    plt.title(f"Compression Rate per Image (JPEG vs PCA) - Quality Factor: {quality_factor}")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.legend()
    plt.savefig(os.path.join(save_folder, f"compression_rate_{quality_factor}.png"))
    plt.close()

In [ ]:
# List of folders to process
all_folders = ['./dataset/chair/', './dataset/Faces', './dataset/kangaroo', './dataset/Motorbikes', './dataset/pizza']
save_base_folder = './comparisons'  # Base folder to save comparison results

# Iterate through each folder and apply the function
for folder in all_folders:
    compare_all(folder, 100, save_base_folder)